In [1]:
import cv2
import numpy as np
import tensorflow as tf


# Load the object classification model

In [ ]:


model = tf.keras.models.load_model('class.h5')

In [ ]:

# Create a background subtractor using the Mixture of Gaussians method
fgbg = cv2.createBackgroundSubtractorMOG2()

# Initialize the camera
cap = cv2.VideoCapture(0)

while True:
    # Capture a frame from the camera
    ret, frame = cap.read()
    
    # Apply the background subtractor to detect movement
    fgmask = fgbg.apply(frame)
    
    # Threshold the difference image to create a binary image
    thresh = cv2.threshold(fgmask, 127, 255, cv2.THRESH_BINARY)[1]
    
    # Find contours in the binary image
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Loop over the contours
    for contour in contours:
        # Calculate the contour area
        area = cv2.contourArea(contour)
        
        # Ignore contours that are too small
        if area < 500:
            continue
        
        # Draw a bounding box around the contour
        x, y, w, h = cv2.boundingRect(contour)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        
        # Extract the ROI (region of interest) from the frame
        roi = frame[y:y+h, x:x+w]
        
        # Preprocess the ROI for classification
        # (resize, convert to grayscale, normalize pixel values, etc.)
        roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        roi = cv2.resize(roi, (64, 64))
        roi = roi / 255.0
        roi = np.expand_dims(roi, axis=-1)
        roi = np.expand_dims(roi, axis=0)
        
        # Classify the ROI using the object classification model
        predictions = model.predict(roi)
        class_id = np.argmax(predictions)
        
        # Print the predicted class label
        print("Predicted class label: {}".format(class_id))
    
    # Show the frame
    cv2.imshow('frame', frame)
    
    # Wait for a key press
    key = cv2.waitKey(1)
    
    # If the 'q' key is pressed, break the loop
    if key == ord('q'):
        break

# Release the camera and close all windows
cap.release()
cv2.destroyAllWindows()
